# Import

In [1]:
import mlflow
import mlflow.keras
import mlflow.tensorflow

import numpy as np
import tensorflow
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

# Setup

In [2]:
mlflow.set_tracking_uri('http://0.0.0.0:5000')

class MlFlowModelCheckpoint(tensorflow.keras.callbacks.ModelCheckpoint):
    def __init__(self, artifact_name, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.artifact_name = artifact_name

    def _save_model(self, epoch, batch, logs):
        super()._save_model(epoch, batch, logs)
        mlflow.keras.log_model(self.model, self.artifact_name)

model_checkpoint_callback = MlFlowModelCheckpoint(
    artifact_name='best_model',
    filepath='../data/checkpoints/best_model.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

params = dict(
    batch_size = 128,
    num_classes = 10,
    epochs = 200,
)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = tf.keras.utils.to_categorical(y_train, params['num_classes'])
y_test = tf.keras.utils.to_categorical(y_test, params['num_classes'])

# Run

In [3]:
def make_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(params['num_classes'], activation='softmax'))
    return model

with mlflow.start_run(run_name='mnist_keras_tutorial') as run:
    mlflow.tensorflow.autolog(
        every_n_iter=1
    )
    
    model = make_model()
    model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.SGD(
                learning_rate=0.00001,
              ),
              metrics=['accuracy'])
    
    history = model.fit(
        x_train,
        y_train,
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        verbose=1,
        validation_data=(x_test, y_test),
        callbacks=[model_checkpoint_callback],
    )

2022-05-30 00:14:44.457655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
469/469 [==============================] - 18s 37ms/step - loss: 2.3785 - accuracy: 0.0765 - val_loss: 2.3587 - val_accuracy: 0.0815
Epoch 2/200
469/469 [==============================] - 16s 34ms/step - loss: 2.3746 - accuracy: 0.0797 - val_loss: 2.3546 - val_accuracy: 0.0834
Epoch 3/200
469/469 [==============================] - 17s 37ms/step - loss: 2.3712 - accuracy: 0.0806 - val_loss: 2.3505 - val_accuracy: 0.0850
Epoch 4/200
469/469 [==============================] - 15s 33ms/step - loss: 2.3657 - accuracy: 0.0832 - val_loss: 2.3464 - val_accuracy: 0.0861
Epoch 5/200
469/469 [==============================] - 16s 34ms/step - loss: 2.3630 - accuracy: 0.0821 - val_loss: 2.3425 - val_accuracy: 0.0889
Epoch 6/200
469/469 [==============================] - 15s 31ms/step - loss: 2.3588 - accuracy: 0.0852 - val_loss: 2.3385 - val_accuracy: 0.0912
Epoch 7/200
469/469 [==============================] - 18s 38ms/step - loss: 2.3545 - accuracy: 0.0878 - val_loss: 2.3346 - val_ac

2022/05/30 00:17:55 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under file:data/mlflow-artifacts/0/3ed5405677b44432b57a94a9e58ced68/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the tracking store. If logging to a mlflow server via REST, consider upgrading the server version to MLflow 1.7.0 or above.


469/469 [==============================] - 18s 39ms/step - loss: 2.3403 - accuracy: 0.0936 - val_loss: 2.3194 - val_accuracy: 0.1063
Epoch 12/200
467/469 [============================>.] - ETA: 0s - loss: 2.3373 - accuracy: 0.0950INFO:tensorflow:Assets written to: /var/folders/90/kskh3wbj04d1cp3_crkb9cgr0000gn/T/tmps_rw1tqn/model/data/model/assets


2022/05/30 00:18:13 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under file:data/mlflow-artifacts/0/3ed5405677b44432b57a94a9e58ced68/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the tracking store. If logging to a mlflow server via REST, consider upgrading the server version to MLflow 1.7.0 or above.


...

2022/05/30 00:29:06 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under file:data/mlflow-artifacts/0/3ed5405677b44432b57a94a9e58ced68/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the tracking store. If logging to a mlflow server via REST, consider upgrading the server version to MLflow 1.7.0 or above.


469/469 [==============================] - 18s 38ms/step - loss: 2.2198 - accuracy: 0.1974 - val_loss: 2.1931 - val_accuracy: 0.2778
Epoch 50/200
465/469 [============================>.] - ETA: 0s - loss: 2.2151 - accuracy: 0.2060INFO:tensorflow:Assets written to: /var/folders/90/kskh3wbj04d1cp3_crkb9cgr0000gn/T/tmphnzkvhrj/model/data/model/assets


2022/05/30 00:29:26 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under file:data/mlflow-artifacts/0/3ed5405677b44432b57a94a9e58ced68/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the tracking store. If logging to a mlflow server via REST, consider upgrading the server version to MLflow 1.7.0 or above.


469/469 [==============================] - 20s 43ms/step - loss: 2.2151 - accuracy: 0.2060 - val_loss: 2.1901 - val_accuracy: 0.2826
Epoch 51/200
467/469 [============================>.] - ETA: 0s - loss: 2.2136 - accuracy: 0.2065INFO:tensorflow:Assets written to: /var/folders/90/kskh3wbj04d1cp3_crkb9cgr0000gn/T/tmpcz1s0oj2/model/data/model/assets


KeyboardInterrupt: 